<a href="https://colab.research.google.com/github/johanjun/Google_Machine_Learning_Bootcamp_KR_2021/blob/main/kaggle/RSNA_MICCAI_BRAIN_TUMOR/rsna_miccai_brain_tumor_johanjun_1015_T1wCE_gridsearch(pre%2C_aug).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install pydicom

     |████████████████████████████████| 2.0 MB 5.2 MB/s 


In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from tqdm import tqdm
import random
import pydicom
import cv2
import pickle

from pydicom.pixel_data_handlers.util import apply_voi_lut

window_center = -500

import matplotlib.pyplot as plt
import glob

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [4]:
data_dir = '/content/drive/MyDrive/Google_Machine_Learning_Bootcamp_KR_2021/RSNA_MICCAI/data/rsna-miccai/'

In [5]:
train_df = pd.read_csv(data_dir + "train_labels.csv")
test_df = pd.read_csv(data_dir + "sample_submission.csv")

In this discussion a competition host has notified that there are some issues with these 3 cases
Patient IDs -

1. 00109 (FLAIR images are blank)
2. 00123 (T1w images are blank)
3. 00709 (FLAIR images are blank)
<br><br>Hence these can be excluded

In [6]:
#refer: https://www.kaggle.com/arnabs007/part-1-rsna-miccai-btrc-understanding-the-data
EXCLUDE = [109, 123, 709]
train_df = train_df[~train_df.BraTS21ID.isin(EXCLUDE)]

In [7]:
TYPES = ["FLAIR", "T1w", "T1wCE", "T2w"] #mpMRI scans

In [8]:
def load_dicom(path, size = 128): #load DICOM files
    dicom = pydicom.read_file(path)
    s = int(dicom.RescaleSlope)
    b = int(dicom.RescaleIntercept)
    data = s * dicom.pixel_array + b

    dicom.WindowCenter = window_center
    dicom.WindowWidth = window_width

    data = apply_voi_lut(data, dicom)

    data = np.clip(data, window_center - (window_width / 2), window_center + (window_width / 2))

    data = dicom.pixel_array #returns a numpy.ndarray containing the pixel data
    if np.max(data) != 0:
        data = data / np.max(data) #standardizes so that the pixel values are between 0 and 1
    data = (data * 255).astype(np.uint8) #rescales to 0 and 255
    return cv2.resize(data, (size, size))

In [9]:
def get_all_image_paths(BraTS21ID, image_type, folder="train"): #get an array of all the images of a particular type or a particular patient id
    assert(image_type in TYPES) #only in types
    patient_path = os.path.join(data_dir + "%s/" % folder, str(BraTS21ID).zfill(5)) #다른 폴더일 수도 있음
    #print(lambda x: int(x[:-4].split("-")[-1]))
    
    paths = sorted(glob.glob(os.path.join(patient_path, image_type, "*")), key=lambda x: int(x[:-4].split("-")[-1])) #sort
    #print(paths)
    
    num_images = len(paths)
    
    start = int(num_images * 0.25)
    end = int(num_images * 0.75)
    if num_images < 10:
        jump = 1
    else:
        jump = 3
        
    return np.array(paths[start:end:jump])

In [10]:
def get_all_images(BraTS21ID, image_type, folder="train", size=225):
    return [load_dicom(path, size) for path in get_all_image_paths(BraTS21ID, image_type, folder)]

In [11]:
def get_all_data_train(image_type):
    global train_df
    
    X = []
    y = []
    train_ids = []

    for i in tqdm(train_df.index):
        tmp_x = train_df.loc[i]
        images = get_all_images(int(tmp_x["BraTS21ID"]), image_type, "train", IMAGE_SIZE)
        label = tmp_x["MGMT_value"]

        X += images
        y += [label] * len(images)
        train_ids += [int(tmp_x["BraTS21ID"])] * len(images)
        assert(len(X) == len(y))
    return np.array(X), np.array(y), np.array(train_ids)

def get_all_data_test(image_type):
    global test_df
    
    X = []
    test_ids = []

    for i in tqdm(test_df.index):
        tmp_x = test_df.loc[i]
        images = get_all_images(int(tmp_x["BraTS21ID"]), image_type, "test", IMAGE_SIZE)
        X += images
        test_ids += [int(tmp_x["BraTS21ID"])] * len(images)

    return np.array(X), np.array(test_ids)

In [12]:
# IMAGE_SIZE = 128
# window_width = 7000

# X_T1wCE, y_T1wCE, train_idt_T1wCE = get_all_data_train("T1wCE")
# X_test_T1wCE, test_idt_T1wCE = get_all_data_test("T1wCE")
# print(X_T1wCE.shape, y_T1wCE.shape, train_idt_T1wCE.shape)

# # #pickling
# with open(data_dir+'X_T1wCE.pkl', 'wb') as f:
#   pickle.dump(X_T1wCE, f)

# with open(data_dir+'y_T1wCE.pkl', 'wb') as f:
#   pickle.dump(y_T1wCE, f)

# with open(data_dir+'train_idt_T1wCE.pkl', 'wb') as f:
#   pickle.dump(train_idt_T1wCE, f)

# with open(data_dir+'X_test_T1wCE.pkl', 'wb') as f:
#   pickle.dump(X_test_T1wCE, f)

# with open(data_dir+'test_idt_T1wCE.pkl', 'wb') as f:
#   pickle.dump(test_idt_T1wCE, f)

# print('T1wCE pickling completed.')

# window_width = 3700

# X_FLAIR, y_FLAIR, train_idt_FLAIR = get_all_data_train("FLAIR")
# X_test_FLAIR, test_idt_FLAIR = get_all_data_test("FLAIR")
# print(X_FLAIR.shape, y_FLAIR.shape, train_idt_FLAIR.shape)

# # #pickling
# with open(data_dir+'X_FLAIR.pkl', 'wb') as f:
#   pickle.dump(X_FLAIR, f)

# with open(data_dir+'y_FLAIR.pkl', 'wb') as f:
#   pickle.dump(y_FLAIR, f)

# with open(data_dir+'train_idt_FLAIR.pkl', 'wb') as f:
#   pickle.dump(train_idt_FLAIR, f)

# with open(data_dir+'X_test_FLAIR.pkl', 'wb') as f:
#   pickle.dump(X_test_FLAIR, f)

# with open(data_dir+'test_idt_FLAIR.pkl', 'wb') as f:
#   pickle.dump(test_idt_FLAIR, f)

# print('FLAIR pickling completed.')

# window_width = 2500

# X_T2w, y_T2w, train_idt_T2w = get_all_data_train("T2w")
# X_test_T2w, test_idt_T2w = get_all_data_test("T2w")
# print(X_T2w.shape, y_T2w.shape, train_idt_T2w.shape)

# # #pickling
# with open(data_dir+'X_T2w.pkl', 'wb') as f:
#   pickle.dump(X_T2w, f)

# with open(data_dir+'y_T2w.pkl', 'wb') as f:
#   pickle.dump(y_T2w, f)

# with open(data_dir+'train_idt_T2w.pkl', 'wb') as f:
#   pickle.dump(train_idt_T2w, f)

# with open(data_dir+'X_test_T2w.pkl', 'wb') as f:
#   pickle.dump(X_test_T2w, f)

# with open(data_dir+'test_idt_T2w.pkl', 'wb') as f:
#   pickle.dump(test_idt_T2w, f)

# print('T2w pickling completed.')

# window_width = 3000

# X_T1w, y_T1w, train_idt_T1w = get_all_data_train("T1w")
# X_test_T1w, test_idt_T1w = get_all_data_test("T1w")
# print(X_T1w.shape, y_T1w.shape, train_idt_T1w.shape)

# # #pickling
# with open(data_dir+'X_T1w.pkl', 'wb') as f:
#   pickle.dump(X_T1w, f)

# with open(data_dir+'y_T1w.pkl', 'wb') as f:
#   pickle.dump(y_T1w, f)

# with open(data_dir+'train_idt_T1w.pkl', 'wb') as f:
#   pickle.dump(train_idt_T1w, f)

# with open(data_dir+'X_test_T1w.pkl', 'wb') as f:
#   pickle.dump(X_test_T1w, f)

# with open(data_dir+'test_idt_T1w.pkl', 'wb') as f:
#   pickle.dump(test_idt_T1w, f)

# print('T1w pickling completed.')

In [13]:
#pickling
with open(data_dir+'X_T1wCE.pkl', 'rb') as f:
  X_T1wCE = pickle.load(f)

with open(data_dir+'y_T1wCE.pkl', 'rb') as f:
  y_T1wCE = pickle.load(f)

with open(data_dir+'train_idt_T1wCE.pkl', 'rb') as f:
  train_idt_T1wCE = pickle.load(f)

with open(data_dir+'X_test_T1wCE.pkl', 'rb') as f:
  X_test_T1wCE = pickle.load(f)

with open(data_dir+'test_idt_T1wCE.pkl', 'rb') as f:
  test_idt_T1wCE = pickle.load(f)
#pickling
with open(data_dir+'X_FLAIR.pkl', 'rb') as f:
  X_FLAIR = pickle.load(f)

with open(data_dir+'y_FLAIR.pkl', 'rb') as f:
  y_FLAIR = pickle.load(f)

with open(data_dir+'train_idt_FLAIR.pkl', 'rb') as f:
  train_idt_FLAIR = pickle.load(f)

with open(data_dir+'X_test_FLAIR.pkl', 'rb') as f:
  X_test_FLAIR = pickle.load(f)

with open(data_dir+'test_idt_FLAIR.pkl', 'rb') as f:
  test_idt_FLAIR = pickle.load(f)
#pickling
with open(data_dir+'X_T2w.pkl', 'rb') as f:
  X_T2w = pickle.load(f)

with open(data_dir+'y_T2w.pkl', 'rb') as f:
  y_T2w = pickle.load(f)

with open(data_dir+'train_idt_T2w.pkl', 'rb') as f:
  train_idt_T2w = pickle.load(f)

with open(data_dir+'X_test_T2w.pkl', 'rb') as f:
  X_test_T2w = pickle.load(f)

with open(data_dir+'test_idt_T2w.pkl', 'rb') as f:
  test_idt_T2w = pickle.load(f)
#pickling
with open(data_dir+'X_T1w.pkl', 'rb') as f:
  X_T1w = pickle.load(f)

with open(data_dir+'y_T1w.pkl', 'rb') as f:
  y_T1w = pickle.load(f)

with open(data_dir+'train_idt_T1w.pkl', 'rb') as f:
  train_idt_T1w = pickle.load(f)

with open(data_dir+'X_test_T1w.pkl', 'rb') as f:
  X_test_T1w = pickle.load(f)

with open(data_dir+'test_idt_T1w.pkl', 'rb') as f:
  test_idt_T1w = pickle.load(f)

In [14]:
# X = np.concatenate([X_T1wCE, X_T2w], axis=0)
# y = np.concatenate([y_T1wCE, y_T2w], axis=0)
# train_idt = np.concatenate([train_idt_T1wCE, train_idt_T2w], axis=0)



# X_test = np.concatenate([X_test_FLAIR, X_test_T1w, X_test_T1wCE, X_test_T2w], axis=0)
# test_idt = np.concatenate([test_idt_FLAIR, test_idt_T1w, test_idt_T1wCE, test_idt_T2w], axis=0)

In [15]:
from tensorflow.keras import layers

data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])

In [16]:
X_train, X_valid, y_train, y_valid, train_idt_train, train_idt_valid = train_test_split(X_T1wCE, y_T1wCE, train_idt_T1wCE, test_size=0.05, random_state=42)

X_train_exp = tf.expand_dims(X_train, -1)
augmented_image = data_augmentation(X_train_exp)
X_train_sq = tf.squeeze(augmented_image)

X_train = np.concatenate([X_train, X_train_sq], axis=0)
y_train = np.concatenate([y_train, y_train], axis=0)

train_idt_train = np.concatenate([train_idt_train, train_idt_train], axis=0)

In [17]:
X_train = tf.expand_dims(X_train, axis=-1) #expand the dimension at the end of the array
X_valid = tf.expand_dims(X_valid, axis=-1)
X_test = tf.expand_dims(X_test_T1wCE, axis=-1)

y_train = to_categorical(y_train) #one-hot incoding
y_valid = to_categorical(y_valid)

In [18]:
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape, train_idt_train.shape, train_idt_valid.shape

(TensorShape([30772, 128, 128, 1]),
 (30772, 2),
 TensorShape([810, 128, 128, 1]),
 (810, 2),
 (30772,),
 (810,))

In [19]:
X2 = np.concatenate([X_FLAIR, X_T2w, X_T1w, X_T1wCE], axis=0)
y2 = np.concatenate([y_FLAIR, y_T2w, y_T1w, y_T1wCE], axis=0)
train_idt2 = np.concatenate([train_idt_FLAIR, train_idt_T2w, train_idt_T1w, train_idt_T1wCE], axis=0)

In [20]:
X_train2, X_valid2, y_train2, y_valid2, train_idt_train2, train_idt_valid2 = train_test_split(X2, y2, train_idt2, test_size=0.05)

In [27]:
X_train2 = np.expand_dims(X_train2, axis=-1) 
X_valid2 = np.expand_dims(X_valid2, axis=-1)

y_train2 = to_categorical(y_train2) 
y_valid2 = to_categorical(y_valid2)

In [29]:
# %load train_lib.py

def train():
    import wandb
    config_defaults = {
        'N_EPOCHS' : 15,
        # 'N_BATCH' : 32,
        # 'DENSE' : 256,
        # 'DROUPOUT' : 0.1
    }
    wandb.init(project='RSNA_1015_T1wCE_augmented',
               config=config_defaults,
               magic=True)

    config = wandb.config
    

    IMAGE_SIZE = 128
    N_EPOCHS = config.N_EPOCHS
    # N_BATCH = config.N_BATCH
    N_BATCH = 32
    N_TRAIN = X_train.shape[0]
    N_VAL = X_valid.shape[0]
    steps_per_epoch = N_TRAIN / N_BATCH
    validation_steps = int(np.ceil(N_VAL / N_BATCH))

    def create_model():
      model = tf.keras.Sequential()
      model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='VALID', 
                                    input_shape=(IMAGE_SIZE, IMAGE_SIZE, 1)))
      model.add(tf.keras.layers.BatchNormalization())
      model.add(tf.keras.layers.MaxPool2D(padding='SAME'))
      
      model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='SAME'))
      model.add(tf.keras.layers.BatchNormalization())
      model.add(tf.keras.layers.MaxPool2D(padding='SAME'))

      model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='SAME'))
      model.add(tf.keras.layers.BatchNormalization())
      model.add(tf.keras.layers.MaxPool2D(padding='SAME'))

      model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='VALID'))
      model.add(tf.keras.layers.BatchNormalization())
      model.add(tf.keras.layers.MaxPool2D(padding='SAME'))
      

      model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='SAME'))
      model.add(tf.keras.layers.BatchNormalization())
      model.add(tf.keras.layers.MaxPool2D(padding='SAME'))
      model.add(tf.keras.layers.Dropout(0.3))

      model.add(tf.keras.layers.Flatten())
      model.add(tf.keras.layers.Dense(256, activation='relu'))
      model.add(tf.keras.layers.Dense(2, activation='softmax'))
      return model
    
    model = create_model()
    

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=[tf.keras.metrics.AUC()])
    
    model.fit(
    x=X_train,
    y=y_train,
    epochs=N_EPOCHS,
    steps_per_epoch=steps_per_epoch,
    validation_data=(X_valid2, y_valid2))

In [30]:
sweep_config = {
    'method': 'grid',
    'parameters': {
        'N_EPOCHS': {
            'values': [15, 40, 65, 90, 150]
        },
        # 'N_BATCH': {
        #     'values': [32, 128]
        # },
        # 'DENSE': {
        #     'values': [128, 512]
        # },
        # 'DROPOUT': {
        #     'values': [0.1, 0.4]
        # },
    }
}

In [31]:
%pip install -q wandb

In [24]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [32]:
import wandb
sweep_id = wandb.sweep(sweep_config, project='RSNA_1015_T1wCE_augmented')

Create sweep with ID: jg3dhrxm
Sweep URL: https://wandb.ai/johanjunme/RSNA_1015_T1wCE_augmented/sweeps/jg3dhrxm


In [33]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: xo6tqr89 with config:
wandb: 	N_EPOCHS: 15
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Epoch 1/15
961/961 [==============================] - 16s 16ms/step - loss: 0.7098 - auc: 0.5728 - val_loss: 0.6772 - val_auc: 0.6015
Epoch 2/15
961/961 [==============================] - 14s 15ms/step - loss: 0.6792 - auc: 0.5988 - val_loss: 0.6827 - val_auc: 0.5875
Epoch 3/15
961/961 [==============================] - 14s 15ms/step - loss: 0.6727 - auc: 0.6124 - val_loss: 0.6838 - val_auc: 0.6144
Epoch 4/15
961/961 [==============================] - 14s 15ms/step - loss: 0.6646 - auc: 0.6249 - val_loss: 0.6784 - val_auc: 0.5978
Epoch 5/15
961/961 [==============================] - 14s 15ms/step - loss: 0.6547 - auc: 0.6402 - val_loss: 0.6635 - val_auc: 0.6260
Epoch 6/15
961/961 [==============================] - 14s 15ms/step - loss: 0.6377 - auc: 0.6660 - val_loss: 0.6781 - val_auc: 0.6125
Epoch 7/15
961/961 [==============================] - 14s 15ms/step - loss: 0.6135 - auc: 0.7018 - val_loss: 0.6482 - val_auc: 0.6600
Epoch 8/15
961/961 [==============================] - 14s 15ms

auc,▁▂▂▂▂▃▄▄▅▆▆▇▇██
epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▇▇▇▇▇▆▆▅▄▃▃▂▂▁
val_auc,▂▁▂▂▃▂▅▃▅▇▇▆▇██
val_loss,▂▂▂▂▁▂▁▂▄▂▂▃▄█▅
auc,0.91904
best_epoch,6
best_val_loss,0.64821
epoch,14
loss,0.35024
val_auc,0.7288


wandb: Agent Starting Run: 83qfku0q with config:
wandb: 	N_EPOCHS: 40
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Epoch 1/40
961/961 [==============================] - 16s 15ms/step - loss: 0.7145 - auc: 0.5656 - val_loss: 0.6801 - val_auc: 0.5993
Epoch 2/40
961/961 [==============================] - 14s 15ms/step - loss: 0.6813 - auc: 0.5914 - val_loss: 0.6868 - val_auc: 0.6030
Epoch 3/40
961/961 [==============================] - 14s 15ms/step - loss: 0.6756 - auc: 0.6059 - val_loss: 0.7017 - val_auc: 0.6158
Epoch 4/40
961/961 [==============================] - 14s 15ms/step - loss: 0.6669 - auc: 0.6183 - val_loss: 0.6894 - val_auc: 0.5895
Epoch 5/40
961/961 [==============================] - 14s 15ms/step - loss: 0.6555 - auc: 0.6363 - val_loss: 0.6866 - val_auc: 0.6222
Epoch 6/40
961/961 [==============================] - 14s 15ms/step - loss: 0.6419 - auc: 0.6603 - val_loss: 0.6723 - val_auc: 0.6373
Epoch 7/40
961/961 [==============================] - 14s 15ms/step - loss: 0.6252 - auc: 0.6844 - val_loss: 0.6691 - val_auc: 0.6034
Epoch 8/40
961/961 [==============================] - 14s 15ms

...

auc,▁▂▂▃▄▅▆▇▇▇▇█████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,██▇▇▆▆▅▄▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_auc,▃▁▃▄▆▆▆█▇▇▇▇█▇▇█▇▇▇▇███▇▇█▇▇▇█▇▇▇███▇▇█▇
val_loss,▁▁▁▁▁▂▁▂▃▃▃▄▃▄▄▄▄▄▅▆▄▅▆▆▆▆▆▆▇▅▆▇▇▇▇▆▇█▇▇
auc,0.99857
best_epoch,1
best_val_loss,0.69116
epoch,89
loss,0.03681
val_auc,0.69989


wandb: Agent Starting Run: 4pori554 with config:
wandb: 	N_EPOCHS: 150
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Epoch 1/150
961/961 [==============================] - 16s 15ms/step - loss: 0.7114 - auc: 0.5746 - val_loss: 0.6712 - val_auc: 0.6196
Epoch 2/150
961/961 [==============================] - 14s 14ms/step - loss: 0.6818 - auc: 0.5913 - val_loss: 0.6791 - val_auc: 0.6071
Epoch 3/150
961/961 [==============================] - 14s 15ms/step - loss: 0.6766 - auc: 0.6051 - val_loss: 0.6731 - val_auc: 0.6154
Epoch 4/150
961/961 [==============================] - 14s 14ms/step - loss: 0.6687 - auc: 0.6183 - val_loss: 0.6778 - val_auc: 0.6053
Epoch 5/150
961/961 [==============================] - 14s 14ms/step - loss: 0.6626 - auc: 0.6269 - val_loss: 0.6727 - val_auc: 0.6200
Epoch 6/150
961/961 [==============================] - 14s 14ms/step - loss: 0.6507 - auc: 0.6469 - val_loss: 0.7029 - val_auc: 0.6183
Epoch 7/150
961/961 [==============================] - 14s 14ms/step - loss: 0.6371 - auc: 0.6651 - val_loss: 0.6825 - val_auc: 0.6087
Epoch 8/150
961/961 [==============================] - 

auc,▁▁▃▅▇▇██████████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,██▇▆▅▄▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_auc,▁▁▂▆▇█▇█▇▇▇▇▇▇▇▇▇▇▇▇▆▆▇▇▇▆▇▇▇▆▇▆▆▇▆▇▇▇▇▆
val_loss,▁▁▁▁▁▁▂▃▃▃▃▄▄▄▄▄▄▅▄▅▅▆▆▆▅▆▆▅▆▆▆▆▆▆█▆▅▆▇▇
auc,0.99922
best_epoch,10
best_val_loss,0.66706
epoch,149
loss,0.02288
val_auc,0.70861


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
